# PDFドキュメントの読込

# 準備

In [1]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv

# 環境変数の読み込み
load_dotenv()
os.environ['OPENAI_API_KEY']  = os.environ['API_KEY']

# Indexの構築

In [2]:
from pathlib import Path
from llama_index import download_loader

# PDFReaderの取得
PDFReader = download_loader("PDFReader")

# PDFドキュメントからテキスト情報を読込
loader = PDFReader()
documents = loader.load_data(file=Path('./data2/001615363.pdf'))

In [3]:
from llama_index.text_splitter import SentenceSplitter
import tiktoken

# NodeParserの作成
node_parser = SentenceSplitter(
    separator="。",
    chunk_size=1024,
    chunk_overlap=20,
    tokenizer=tiktoken.encoding_for_model("gpt-3.5-turbo-1106").encode)

In [4]:
from llama_index.llms import OpenAI
from llama_index import ServiceContext
from llama_index import VectorStoreIndex

# 言語モデルの指定
llm = OpenAI(model='gpt-3.5-turbo-1106', temperature=1.2)

system_prompt = """\
あなたは世界中で信頼されているエキスパート Q&A システムです。\n事前知識ではなく、常に提供されたコンテキスト情報を使用して質問に答えてください。\n従うべきいくつかのルール:\n1.回答内で指定されたコンテキストを直接参照しないでください。\n2. 「コンテキストに基づいて、...」や「コンテキスト情報は...」、またはそれに類するような記述は避けてください。
"""

# サービスコンテキストの作成
service_context = ServiceContext.from_defaults(
  system_prompt=system_prompt,
  llm=llm, 
  node_parser=node_parser)

# Indexの構築
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [5]:
# 参考：言語モデルのエンコーディングを調べる
import tiktoken
print(tiktoken.encoding_for_model('gpt-3.5-turbo-1106').encode)

<bound method Encoding.encode of <Encoding 'cl100k_base'>>


# Query Engineの作成

In [6]:
# Query Engineの作成
query_engine = index.as_query_engine(
    service_context=service_context,
    similarity_top_k=3,
    streaming=True,
)

# ユーザーからの質問に回答

In [10]:
response = query_engine.query("お土産の購入状況を教えて")
# 言語モデルからの回答を表示
response.print_response_stream()

お土産の購入状況は、特定の国籍や地域によって異なります。一般的には「菓子類」が最も多く購入されており、特に韓国、台湾、香港、中国の旅行者によって多く購入されています。また、消費者ごとの平均支出については、台湾や香港地域の旅行者が「宝石・貴金属」に最も多くの支出をしていることが報告されています。さらに、消費税免税手続きを実施する割合も国籍や地域によって大きく異なり、台湾と香港地域では特に高い割合で免税手続きが行われていることが明らかになっています。

In [11]:
# 出典を表示
print(response.get_formatted_sources())

> Source (Node id: c7ae1f11-4f05-478a-960f-441bc92630ff): 訪日外国人消費動向調査  
 
20 ３．土産品の購入実態  
 
（１） 費目別購入率 
 費目別の 購入率（その費目を購入した人の
割合）は「菓子類」 （ 70.4％）、「その他食
料品・...

> Source (Node id: e843b8f8-896c-4890-a2bb-0c75be19edee): 訪日外国人消費動向調査 
 
38 
 （３）買物場所 
百貨店・デパート 
原則として百貨店協会加盟の店舗 
家電量販店 
PCやカメラ、電気製品を専門に販売する店舗 
ファッション専門店 
...

> Source (Node id: e3ac7233-19e0-4dba-8285-28c2e27c6a63): 21 （３）消費税免税手続き の実施状況  
 今回の日本滞在中に 消費税免税手続き
を実施した人の割合 は全体の40.3％で
ある（図表 3-3）。 
 国籍･地域別に みると、 台湾と香...


# 詳細なログの表示

In [12]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [13]:
response = query_engine.query("お土産の購入状況を教えて")
# 言語モデルからの回答を表示
print(response)

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'post_parser': <function Embeddings.create.<locals>.parser at 0x000001875B169620>, 'json_data': {'input': ['お土産の購入状況を教えて'], 'model': <OpenAIEmbeddingModeModel.TEXT_EMBED_ADA_002: 'text-embedding-ada-002'>, 'encoding_format': 'base64'}}
Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'post_parser': <function Embeddings.create.<locals>.parser at 0x000001875B169620>, 'json_data': {'input': ['お土産の購入状況を教えて'], 'model': <OpenAIEmbeddingModeModel.TEXT_EMBED_ADA_002: 'text-embedding-ada-002'>, 'encoding_format': 'base64'}}
DEBUG:httpcore.connection:close.started
close.started
DEBUG:httpcore.connection:close.complete
close.complete
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=60.0 socket_options=None
connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=60.0 socket_options=None
DEBUG:httpcore

In [18]:
# もとに戻す（ログを出力しない）
logging.basicConfig(stream=sys.stdout, level=logging.WARNING)
# ログハンドラー出力を除去
while len(logging.getLogger().handlers) > 0:
  logging.getLogger().removeHandler(logging.getLogger().handlers[0])

# ストレージに保存

In [15]:
# ストレージに保存
index.storage_context.persist("./storage02")

In [16]:
from llama_index import StorageContext, load_index_from_storage

# ストレージコンテキストの作成
storage_context = StorageContext.from_defaults(persist_dir="./storage02")
# Indexのロード
index = load_index_from_storage(storage_context)

In [17]:
response = query_engine.query("お土産の購入状況を教えて")
# 言語モデルからの回答を表示
print(response)

お土産として購入される費目の購入率や購入者単価についての情報が提供されています。菓子類が最も購入率が高く、その他食料品・飲料・たばこ、衣類も高い購入率を示しています。国籍や地域によっても購入率に違いがあり、例えば韓国、台湾、香港、中国では菓子類が特に高い購入率を示し、米国ではその他食料品・飲料・たばこの購入率が高いことがわかります。また、宝石・貴金属の購入者単価が特に高いことも示されています。
